In [0]:
users_data_df = spark.read.format("delta").load("/mnt/adls21s/silver/users_data/")
cards_data_df = spark.read.format("delta").load("/mnt/adls21s/silver/cards_data/")
transactions_data_df = spark.read.format("delta").load("/mnt/adls21s/silver/transactions_data/")